# 작업 계획
- 1) 라이브러리 로드
- 2) 데이터 로드 및 전처리 (NaN 정리, 칼럼 정리, 칼럼 재명명)
- 3) LSTM 모델 정비 및 가동
- 4) 모델 성능 확인
- 5) 함수 모듈화 진행

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.models import load_model
import keras
import h5py
import requests
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data = pd.read_excel("OfficialDatasetAfter00.xlsx")

In [4]:
data.head()

,연,월,일,시간,요일,윤년,전력부하,전력부하 예측치,신정,설연휴,...,적설량(서울),적설량(대전),적설량(대구),적설량(광주),적설량(부산),풍속(서울),풍속(대전),풍속(대구),풍속(광주),풍속(부산)
0,year,month,date,time,day,lunar,elec,elecfc,sd1,sd2_hol,...,snow_su,snow_dj,snow_dg,snow_gj,snow_bs,wind_su,wind_dj,wind_dg,wind_gj,wind_bs
1,2000,1,1,0,6,1,23415.8,NaN,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,1,1,1,6,1,22352.4,23500,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,1,1,2,6,1,21428.2,22520,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,1,1,3,6,1,20723.9,21810,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['연', '월', '일', '시간', '요일', '윤년', '전력부하', '전력부하 예측치', '신정', '설연휴', '설날',
       '삼일절', '식목일', '노동절', '석가탄신일', '어린이날', '현충일', '제헌절', '광복절', '개천절', '추석',
       '추석연휴', '국군의 날', '한글날', '크리스마스', '대선', '국회의원선거', '지방선거', '임시공휴일',
       '대체공휴일', '연휴', '징검다리일', '징검다리연휴', '여름 휴가', '기온(서울)', '기온(대전)', '기온(대구)',
       '기온(광주)', '기온(부산)', '습도(서울)', '습도(대전)', '습도(대구)', '습도(광주)', '습도(부산)',
       '강수량(서울)', '강수량(대전)', '강수량(대구)', '강수량(광주)', '강수량(부산)', '적설량(서울)',
       '적설량(대전)', '적설량(대구)', '적설량(광주)', '적설량(부산)', '풍속(서울)', '풍속(대전)',
       '풍속(대구)', '풍속(광주)', '풍속(부산)'],
      dtype='object')

In [6]:
data.describe()

,연,월,일,시간,요일,윤년,전력부하,전력부하 예측치,신정,설연휴,...,적설량(서울),적설량(대전),적설량(대구),적설량(광주),적설량(부산),풍속(서울),풍속(대전),풍속(대구),풍속(광주),풍속(부산)
count,160681,160681,160681,160681,160681,160681,160297,160608,160681,160681,...,1,1,1,1,1,1,1,1,1,1
unique,20,13,32,25,8,3,68952,23624,3,5,...,1,1,1,1,1,1,1,1,1,1
top,2000,3,16,12,7,0,50541,48400,0,0,...,snow_su,snow_dj,snow_dg,snow_gj,snow_bs,wind_su,wind_dj,wind_dg,wind_gj,wind_bs
freq,8784,14136,5280,6695,22968,116760,13,277,160224,158905,...,1,1,1,1,1,1,1,1,1,1


In [10]:
sel_time = ['연','월','일','시간','요일','전력부하']
sel_time_temp = ['연','월','일','시간','요일','전력부하',
                 '기온(서울)', '기온(대전)', '기온(대구)','기온(광주)', '기온(부산)']
sel_time_temp_humid = ['연','월','일','시간','요일','전력부하',
                 '기온(서울)', '기온(대전)', '기온(대구)','기온(광주)', '기온(부산)'
                      '습도(서울)', '습도(대전)', '습도(대구)', '습도(광주)', '습도(부산)']

In [12]:
d_time = data[sel_time]

In [15]:
d_time.rename(columns = {'연':'Year','월':'Month','일':'Day','시간':'24H',
               '요일':'Weekdays','전력부하':'Elec'}, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [16]:
d_time.sample(5)

,Year,Month,Day,24H,Weekdays,Elec
97241,2011,2,3,16,4,38036
127614,2014,7,23,5,3,52303
1741,2000,3,13,12,1,30766
22512,2002,7,26,23,5,36077.8
139619,2015,12,5,10,6,62921


In [17]:
elec_series = d_time['Elec']

In [58]:
d_time.isnull()

,Year,Month,Day,24H,Weekdays,Elec
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [60]:
d_time[d_time.isnull().any(axis=1)] # 

,Year,Month,Day,24H,Weekdays,Elec
160297,2018,4,15,0,7,NaN
160298,2018,4,15,1,7,NaN
160299,2018,4,15,2,7,NaN
160300,2018,4,15,3,7,NaN
160301,2018,4,15,4,7,NaN
160302,2018,4,15,5,7,NaN
160303,2018,4,15,6,7,NaN
160304,2018,4,15,7,7,NaN
160305,2018,4,15,8,7,NaN
160306,2018,4,15,9,7,NaN


In [63]:
d_one = d_time.copy()
d_one = d_one[:160297]

In [64]:
d_one.isnull().any(axis=1)

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
160267    False
160268    False
160269    False
160270    False
160271    False
160272    False
160273    False
160274    False
160275    False
160276    False
160277    False
160278    False
160279    False
160280    False
160281    False
160282    False
160283    False
160284    False
160285    False
160286    False
160287    False
160288    False
160289    False
160290    False
160291    False
160292    False
160293    False
160294    False
160295    False
160296    False
Length: 160297, dtype: b

# 대상 데이터 d_one 
## - 2000년 1월 1일부터 2018년 4월 14일까지
## - 햇수로 19....
## - 00년부터 15년까지 train, 16,17년 val, 18년 3월 1일 예측

In [65]:
d_one.tail() 

,Year,Month,Day,24H,Weekdays,Elec
160292,2018,4,14,19,6,58765
160293,2018,4,14,20,6,58413
160294,2018,4,14,21,6,57790
160295,2018,4,14,22,6,57964
160296,2018,4,14,23,6,58269


In [66]:
d_one.head() 

,Year,Month,Day,24H,Weekdays,Elec
0,year,month,date,time,day,elec
1,2000,1,1,0,6,23415.8
2,2000,1,1,1,6,22352.4
3,2000,1,1,2,6,21428.2
4,2000,1,1,3,6,20723.9


In [68]:
d_one.drop(index = 0, axis = 0, inplace = True)

In [69]:
d_one.head()

,Year,Month,Day,24H,Weekdays,Elec
1,2000,1,1,0,6,23415.8
2,2000,1,1,1,6,22352.4
3,2000,1,1,2,6,21428.2
4,2000,1,1,3,6,20723.9
5,2000,1,1,4,6,20225.3


In [70]:
len(d_one)

160296

In [72]:
160296/24 #6679일 데이터, 160296 시간 데이터

6679.0

In [73]:
def load_data(stock, seq_len):
    amount_of_features = len(stock.columns) # 5
    data = stock.as_matrix() 
    sequence_length = seq_len + 1 # index starting from 0
    result = []
    
    for index in range(len(data) - sequence_length): # maxmimum date = lastest date - sequence length
        result.append(data[index: index + sequence_length]) # index : index + 22days
    
    result = np.array(result)
    row = round(0.9 * result.shape[0]) # 90% split
    train = result[:int(row), :] # 90% date, all features 
    
    x_train = train[:, :-1] 
    y_train = train[:, -1][:,-1]
    
    x_test = result[int(row):, :-1] 
    y_test = result[int(row):, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

In [74]:
def build_model(layers):
    d = 0.3
    model = Sequential()
    
    model.add(LSTM(256, input_shape=(layers[1], layers[0]), return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(256, input_shape=(layers[1], layers[0]), return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="uniform",activation='relu'))        
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))
    
    # adam = keras.optimizers.Adam(decay=0.2)
        
    start = time.time()
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model




In [77]:
window = 24 # 24시간이니까 나중에 36시간 48시간, 오버랩도 지금의 1 말고 6시간 12시간 24시간
X_train, y_train, X_test, y_test = load_data(d_one, window)
print (X_train[0], y_train[0])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


[[2000 1 1 0 6 23415.8]
 [2000 1 1 1 6 22352.4]
 [2000 1 1 2 6 21428.2]
 [2000 1 1 3 6 20723.9]
 [2000 1 1 4 6 20225.3]
 [2000 1 1 5 6 20103.7]
 [2000 1 1 6 6 20078]
 [2000 1 1 7 6 19898.1]
 [2000 1 1 8 6 19367.6]
 [2000 1 1 9 6 19631.5]
 [2000 1 1 10 6 19936]
 [2000 1 1 11 6 19849.6]
 [2000 1 1 12 6 19591.8]
 [2000 1 1 13 6 19601.4]
 [2000 1 1 14 6 19622.3]
 [2000 1 1 15 6 19732.5]
 [2000 1 1 16 6 20227.2]
 [2000 1 1 17 6 21802.5]
 [2000 1 1 18 6 23041.4]
 [2000 1 1 19 6 23121.3]
 [2000 1 1 20 6 22933.1]
 [2000 1 1 21 6 22947.5]
 [2000 1 1 22 6 24222.7]
 [2000 1 1 23 6 23868.3]] 22219.8


In [80]:
d_one.head(26)

,Year,Month,Day,24H,Weekdays,Elec
1,2000,1,1,0,6,23415.8
2,2000,1,1,1,6,22352.4
3,2000,1,1,2,6,21428.2
4,2000,1,1,3,6,20723.9
5,2000,1,1,4,6,20225.3
6,2000,1,1,5,6,20103.7
7,2000,1,1,6,6,20078
8,2000,1,1,7,6,19898.1
9,2000,1,1,8,6,19367.6
10,2000,1,1,9,6,19631.5


In [81]:
print (X_train[1], y_train[1])

[[2000 1 1 1 6 22352.4]
 [2000 1 1 2 6 21428.2]
 [2000 1 1 3 6 20723.9]
 [2000 1 1 4 6 20225.3]
 [2000 1 1 5 6 20103.7]
 [2000 1 1 6 6 20078]
 [2000 1 1 7 6 19898.1]
 [2000 1 1 8 6 19367.6]
 [2000 1 1 9 6 19631.5]
 [2000 1 1 10 6 19936]
 [2000 1 1 11 6 19849.6]
 [2000 1 1 12 6 19591.8]
 [2000 1 1 13 6 19601.4]
 [2000 1 1 14 6 19622.3]
 [2000 1 1 15 6 19732.5]
 [2000 1 1 16 6 20227.2]
 [2000 1 1 17 6 21802.5]
 [2000 1 1 18 6 23041.4]
 [2000 1 1 19 6 23121.3]
 [2000 1 1 20 6 22933.1]
 [2000 1 1 21 6 22947.5]
 [2000 1 1 22 6 24222.7]
 [2000 1 1 23 6 23868.3]
 [2000 1 2 0 7 22219.8]] 20901


In [82]:
len(X_train)

144244

In [83]:
len(X_test)

16027

In [86]:
model = build_model([6,window,1]) # 각 변수, 5, wind, 1
# build_model(layers)
# layers = [5, window,1] = [5, 24,1]

'''
input_shape=(layers[1], layers[0])
layers[1] = 24, layers[0] = 6 (#of colums)

'''


Compilation Time :  0.023680925369262695


'\ninput_shape=(layers[1], layers[0])\nlayers[1] = 24, layers[0] = 6 (#of colums)\n\n'

In [87]:
#import time
start_time = time.time()

#work
model.fit(X_train,y_train,batch_size=512,epochs=30,validation_split=0.1,verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Train on 129819 samples, validate on 14425 samples
Epoch 1/30
129819/129819 [==============================] - 269s 2ms/step - loss: 2163647443.4067 - acc: 0.0000e+00 - val_loss: 3583943929.3094 - val_acc: 0.0000e+00
Epoch 2/30
129819/129819 [==============================] - 267s 2ms/step - loss: 2038193505.0311 - acc: 0.0000e+00 - val_loss: 3334272982.2769 - val_acc: 0.0000e+00
Epoch 3/30
129819/129819 [==============================] - 271s 2ms/step - loss: 1803384730.9954 - acc: 0.0000e+00 - val_loss: 2959194341.9475 - val_acc: 0.0000e+00
Epoch 4/30
129819/129819 [==============================] - 269s 2ms/step - loss: 1501941868.2408 - acc: 0.0000e+00 - val_loss: 2511881442.2561 - val_acc: 0.0000e+00
Epoch 5/30
129819/129819 [==============================] - 274s 2ms/step - loss: 1177568096.3749 - acc: 0.0000e+00 - val_loss: 2043451594.5284 - val_acc: 0.0000e+00
Epoch 6/30
129819/129819 [==============================] - 267s 2ms/step - loss: 869290598.8193 - acc: 0.0000e+00 - va

In [88]:
def model_score(model, X_train, y_train, X_test, y_test):
    trainScore = model.evaluate(X_train, y_train, verbose=0)
    print('Train Score: %.5f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

    testScore = model.evaluate(X_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]


model_score(model, X_train, y_train, X_test, y_test)

Train Score: 141057026.00288 MSE (11876.74 RMSE)
Test Score: 397303023.69851 MSE (19932.46 RMSE)


(141057026.002884, 397303023.69850874)

In [89]:
import matplotlib.pyplot as plt2

plt2.figure(figsize = (16,9))
plt2.plot(newp,color='red', label='Prediction')
plt2.plot(newy_test,color='blue', label='Actual')
plt2.legend(loc='best')
plt2.show()

NameError: name 'newp' is not defined

<Figure size 1152x648 with 0 Axes>